In [1]:
from transformer_chat_model.chat_models import TransformerChatModel

In [2]:
from transformers import BitsAndBytesConfig
import torch

In [3]:
from langchain_core.messages import HumanMessage 

In [4]:
pretrained_model_name_or_path = "Qwen/Qwen2.5-3B-Instruct"
# Create your quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
chat_model = TransformerChatModel(
    pretrained_model_name_or_path=pretrained_model_name_or_path,
    quantization_config = bnb_config,
    max_new_tokens = 1024,
    device = "auto"
    )

Loading Model ...



2026-02-16 19:51:29.675512: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771271489.706669   99827 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771271489.713894   99827 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771271489.733452   99827 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771271489.733478   99827 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771271489.733483   99827 computation_placer.cc:177] computation placer alr

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
chat_model.debug=True

In [6]:
print(chat_model.invoke("What is 5 + 3 ?").content)

=========================DEBUG - PROMPT=========================
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is 5 + 3 ?<|im_end|>
<|im_start|>assistant



5 + 3 equals 8.


In [7]:
from langchain_core.tools import BaseTool, tool

In [8]:
@tool
def factorial(num: int )-> int:
    """Calculate factorial of a number"""
    res = 1
    for i in range(2,num):
        res*=i
    return res

In [9]:
@tool
def check_haiku_lines(text: str) -> str:
  """
  Check if the haiku has exactly three lines.
  Use this to check a format of haiku whenever you write a haiku.
  """
  lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
  if len(lines) != 3:
      return f"Incorrect! The haiku must have 3 lines. This haiku has {len(lines)} lines."
  else:
      return "Correct! This haiku has 3 lines."
from langchain.agents import create_agent
agent = create_agent(
    model = chat_model,
    tools = [check_haiku_lines, factorial],
)

In [22]:
response = agent.invoke({"messages":"What is factorial of 10 and sum of 3.5 + 7 ?" })

=========================DEBUG - PROMPT=========================
<|im_start|>system
You are a helpful assistant.You have access to the following tools:
**check_haiku_lines**:
	Check if the haiku has exactly three lines.
Use this to check a format of haiku whenever you write a haiku.
	params:
		text (string)

**factorial**:
	Calculate factorial of a number
	params:
		num (integer)

Whenever you need to use a tool, you MUST respond with a JSON code block in this exact format:
```tool_call_block
{
    "tool_calls": [
        {
            "name": tool_name,
            "arguments": {"param1": "value1", "param2": "value2"}
            }
        ]
    }
```
Only use provided tools, do not invent new ones.
If you don't need to use any tools, respond normally without the JSON block.
Tool responses are provided with a tool_call_id.
Always match each responese to the corresponding tool call using this ID and use the matched results to produce the final answer.
A user query may contain multiple 

In [23]:
print(response["messages"][-1].content)

The factorial of 10 is 362880. The sum of 3.5 and 7 is 10.5.


In [15]:
for message in response["messages"]:
    print(message)

content='What is factorial of 10 and sum of 3.5 + 7 ?' additional_kwargs={} response_metadata={} id='19017e97-b240-45e4-9365-8014a191b545'
content='```tool_call_block\n{\n    "tool_calls": [\n        {\n            "tool_name": "factorial",\n            "arguments": {"num": 10}\n        },\n        {\n            "tool_name": "check_math_expression",\n            "arguments": {"expression": "3.5 + 7"}\n        }\n    ]\n}\n```' additional_kwargs={} response_metadata={} id='lc_run--019c6803-1d99-7a72-8a86-80e5d5f34e79-0' tool_calls=[] invalid_tool_calls=[]
